<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Final-Project-Submission" data-toc-modified-id="Final-Project-Submission-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Final Project Submission</a></span></li><li><span><a href="#Initial-Thoughts-and-Response-to-the-Business-Problem" data-toc-modified-id="Initial-Thoughts-and-Response-to-the-Business-Problem-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Initial Thoughts and Response to the Business Problem</a></span></li><li><span><a href="#Importing-Modules-and-Opening-Relevant-DataFrames" data-toc-modified-id="Importing-Modules-and-Opening-Relevant-DataFrames-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Importing Modules and Opening Relevant DataFrames</a></span></li><li><span><a href="#Box-Office-Gross" data-toc-modified-id="Box-Office-Gross-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Box Office Gross</a></span></li><li><span><a href="#Title-Basics" data-toc-modified-id="Title-Basics-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Title Basics</a></span></li><li><span><a href="#Sorting-and-Refining-Data-by-Genre" data-toc-modified-id="Sorting-and-Refining-Data-by-Genre-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Sorting and Refining Data by Genre</a></span></li><li><span><a href="#Exploring-the-Correlation-Between-Critical-and-Commercial-Success" data-toc-modified-id="Exploring-the-Correlation-Between-Critical-and-Commercial-Success-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Exploring the Correlation Between Critical and Commercial Success</a></span></li><li><span><a href="#Examining-ROI-by-Genre" data-toc-modified-id="Examining-ROI-by-Genre-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Examining ROI by Genre</a></span></li><li><span><a href="#For-Further-Explanation" data-toc-modified-id="For-Further-Explanation-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>For Further Explanation</a></span></li></ul></div>

## Final Project Submission

Please fill out:
* Student name: Daniel Ross-Leutwyler
* Student pace: **self paced** / part time / full time
* Scheduled project review date/time: 
* Instructor name: James Irving
* Blog post URL:


## Initial Thoughts and Response to the Business Problem

**"Microsoft sees all the big companies creating original video content and they want to get in on the fun. They have decided to create a new movie studio, but they don’t know anything about creating movies. You are charged with exploring what types of films are currently doing the best at the box office. You must then translate those findings into actionable insights that the head of Microsoft's new movie studio can use to help decide what type of films to create."**

There are lots of metrics to measure success. With the data at hand the two main ways to judge the success of a movie are through analyzing key financial metrics (gross box office, and return-on-investment, for example), and critical response. 

By analyzing what, if any, attributes the top grossing movies all share, we can make informed suggestions about what sort of film Microsoft should pursue.

I will also explore the relationship between genre, budget, and gross revenue. The success of a blockbuster is in part dependent upon its ROI, as well.

The analysis performed here will attempt to answer these three questions:

1) What are the highest grossing genres?

2) What genres have the highest ROI?

3) Does a higher budget necessarily correlate with a higher gross?

## Importing Modules and Opening Relevant DataFrames

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# suppressing scientific notation, and adding ',' to long values for legibility
pd.options.display.float_format = '{:,.2f}'.format
# setting sns context to talk for clarity and size
sns.set_context('talk')

These are the data sets are relevant to answering the stakeholder questions.

In [ ]:
# creating variables to call for opening csv
box_office_gross = 'zippedData/bom.movie_gross.csv'
title_basics = 'zippedData/imdb.title.basics.csv'
ratings_url = 'zippedData/imdb.title.ratings.csv.gz'
budget_url = 'zippedData/tn.movie_budgets.csv'

These data sets are not relevant to the business question, either due to lack of crossover/compatibility with other sets, and outdated or incomplete data.

imdb.name.basics.csv
imbd.title.akas.csv\
imdb.title.crew.csv\
imdb.principles.csv\
rt.movie_info.tsv\
rt.reviews.tsv\
tmdb.movies.csv


In [ ]:
meta_url = 'zippedData/movies_metadata.csv.zip'

## Box Office Gross

Preliminary EDA on bom.movie_gross.csv

In [ ]:
def prelim(df):
    """access core information
    on a dataframe at the beginning of EDA
    """
    return (display(df.head()), 
            display(df.info()), 
            display(df.isna().sum()))

In [ ]:
gross_df = pd.read_csv(box_office_gross)
prelim(gross_df)

gross_df is a data frame of movies from 2011-2018. Each row represents one movie, and each column contains the the following values:

***'title', 'studio', 'domestic_gross', 'foreign_gross', and 'year'.***

There are no null entries in the ***title*** column, which contains objects.

There are 5 null entries in the ***studio*** column, which contains objects.

There are 28 null values in the ***domestic_gross*** column, which contains numbers, specifically floats.

There are 1350 null values in the ***foreign_gross*** column, which contains objects, and will need to be cast as a float. The null values will need to be replaced.

***hypothesis: foreign_gross should be summed with domestic_gross, as the global nature of media today makes the foreign/domestic binary less important. ***

There are no nulls in the ***year*** column, which contains integers. 

Examining the 'foreign_gross' column:

In [ ]:
# removing commas to be able to cast as float and sanity check
gross_df['foreign_gross'] = gross_df['foreign_gross'].str.replace(',','')
gross_df['foreign_gross'] = gross_df['foreign_gross'].astype(float)
gross_df.info()

In [ ]:
# small function to look at the rows with null values
def see_nans(df, cols=None): 
    """accepts a data frame, and optionally columns
    returns a data frame of all null values.
    
    Used for previewing missing data.
    Does not alter df in any way"""
    
    if cols is None:
        cols = df.columns
    return df[df[cols].isnull().any(axis=1)]

see_nans(gross_df, ['foreign_gross']).head(20)

A cursory search of the foreign box office receipts for several movies on this list demonstrates that while some of these movies did not have a foreign theatrical release (Flipped), it appears that some of the movies foreign box office receipts have already been counted in the domestic gross category (Courageous), and others have simply had that info omitted. It is also worth mentioning that the movies missing the foreign_gross data are not on the upper end of the the domestic_gross category, rendering their relevance minimal. I will replace all the Nan values in this column with 0.

In [ ]:
# replacing NaNs with 0 and sanity check
gross_df['foreign_gross'] = gross_df['foreign_gross'].fillna(0)
see_nans(gross_df, ['foreign_gross']).head(20)

In [ ]:
# exploring the difference in domestic and foreign gross
print(f"The domestic gross sum is: ${round(gross_df['domestic_gross'].sum()):,}")
print(f"The domestic gross mean is: ${round(gross_df['domestic_gross'].mean()):,}")
print(f"The foreign gross sum is: ${round(gross_df['foreign_gross'].sum()):,}")
print(f"The foreign gross mean is: ${round(gross_df['foreign_gross'].mean()):,}")

Above we can see that the both the total foreign box office receipts (even with the 1350 replaced data points) and the foreign box office mean are ***higher*** than domestic.

This means it ***may*** warrant giving special consideration to movies that performed well in foreign markets.

I will also add an additional column for total_gross summing the domestic and foreign columns, as this is a feature that is absent in the original data.

In [ ]:
gross_df['total_gross'] = (gross_df['domestic_gross'] + gross_df['foreign_gross'])
gross_df.head()

In [ ]:
# breaking out new dfs to explore how domestic, foreign, and total compare
top_100_domestic = gross_df.sort_values('domestic_gross', ascending=False)[:100]
top_100_foreign = gross_df.sort_values('foreign_gross', ascending=False)[:100]
top_100_total = gross_df.sort_values('total_gross', ascending=False)[:100]

display(top_100_domestic.head())
display(top_100_foreign.head())
display(top_100_total.head())

It is clear from looking at the top_100_domestic films sorted by domestic_gross (top table) that the foreign_gross values are incorrect. It is not possible that there were only $1,131 of receipts for Star Wars the Force Awakens, and imdb stats confirm this.

Exploring incorrect values in the foreign_gross column:

In [ ]:
top_100_domestic.sort_values('foreign_gross').head(6)

Ok, luckily we only have to manually scrape and replace the foreign gross box office receipts from 5 movies. Phew.

In [ ]:
# creating dictionary to replace incorrect values.
# correct values taken from IMBD

mapping_dict = {1010.00 : 1009996733,
               1019.40 : 1018130819,
               1131.60 : 1132859475,
               1163.00 : 1162334379,
               1369.50 : 1369544272}

gross_df['foreign_gross'] = gross_df['foreign_gross'].replace(mapping_dict)

# also need to re-concatinate the total_gross to reflect the updated foreign_gross

gross_df['total_gross'] = (gross_df['domestic_gross'] + gross_df['foreign_gross'])

# recreating the top_100_x dataframes with the updated
# foreign receipt data

top_100_domestic = gross_df.sort_values('domestic_gross', ascending=False)[:100]
top_100_foreign = gross_df.sort_values('foreign_gross', ascending=False)[:100]
top_100_total = gross_df.sort_values('total_gross', ascending=False)[:100]

display(top_100_domestic.head(10))
display(top_100_foreign.head(10))
display(top_100_total.head(10))

In [ ]:
fig, [ax1, ax2, ax3] = plt.subplots(1 ,3 , figsize=(20,10))

sns.barplot(data=top_100_domestic.head(10), x='title', y='domestic_gross', ax=ax1)
sns.barplot(data=top_100_foreign.head(10), x='title', y='foreign_gross', ax=ax2)
sns.barplot(data=top_100_total.head(10), x='title', y='total_gross', ax=ax3)

ax1.set_title('Domestic')
ax1.set_ylabel('hundred millions')
ax1.set_xticklabels(ax1.get_xticklabels(),rotation=45, ha='right')

ax2.set_title('Foreign')
ax2.set_ylabel('billions')
ax2.set_xticklabels(ax2.get_xticklabels(),rotation=45, ha='right')

ax3.set_title('Total')
ax3.set_ylabel('billions')
ax3.set_xticklabels(ax3.get_xticklabels(),rotation=45, ha='right')

plt.tight_layout()

plt.suptitle('Top 5 Movies by Domestic, Foreign, and Total Gross', y=1.1);

The only conclusion that we can draw from the selected data is that movies that are in a series have the highest gross, in both the foreign and domestic categories. In fact, out of the top 10 by total sales only one movie, Black Panther, is not part of a series.

Combining this data frame with another that contains info about genre will allow for more direct suggestions to be made.

## Title Basics

Preliminary EDA on imdb.title.basics.csv

In [ ]:
title_basics_df = pd.read_csv(title_basics)
prelim(title_basics_df)

The title.basics table appears to contain a data frame primarily of movies from 2010-2018, but also with some titles from ***THE FUTURE***. Each row represents one movie, and contains columns with the following values:

'tconst', 'primary_title', 'original_title', 'start_year','runtime_minutes', 'genres'

There are no null entries in the tconst column, which is an object, as I would expect, and this column can be used as the index to ***join this df with other dfs that are similarly formatted.***

There are no null entries in the start_year column, which is an integer, as I would expect. ***This column will require cleaning to deal with movies from the future.***

There are significant null entires in the runtime_minutes column, which may be of questionable use. ***It may be worth exploring the relationship between movie length and box office success.***

There are 5408 null values in the genres column. This column will require more exploration as we will need it to help make decisions about what kind of movies Microsoft should be making.

Exploring the 'start_year' column:

In [ ]:
title_basics_df['start_year'].value_counts()

In [ ]:
title_basics_df[title_basics_df['start_year'] > 2021].head()

In [ ]:
# we will set aside the movies from the future. There may be vaule in
# evaluating them by genre to see what is lined up in the future, but
# this evaluation would be supplementary to our primary question.
future_movies_df = title_basics_df.loc[title_basics_df['start_year'] > 2021]
title_basics_df = title_basics_df.loc[title_basics_df['start_year'] <= 2021]

# sanity check
title_basics_df['start_year'].value_counts()

## Sorting and Refining Data by Genre

At this point, to limit the scope of the data to evaluate, the title_basics_df and the gross_df should be merged.

In [ ]:
display(title_basics_df.head())
display(gross_df.head())

There are two potential columns in the title_basics_df that we can use to merge: 'primary_title' and 'original_title'. Below are two different dfs that show us which column has more overlap with the gross_df.

In [ ]:
df_orig = pd.merge(title_basics_df, gross_df, left_on='original_title',
              right_on='title')
df_pri = pd.merge(title_basics_df, gross_df, left_on='primary_title',
              right_on='title')
display(len(df_orig))
display(len(df_pri))

The 'primary_title' series has more overlap with gross_df. This is the merged df that we will use for the rest of our exploration.

Exploring the 'genre' column:

In [ ]:
df_pri['genres'].value_counts()

In [ ]:
# splitting the genres into lists in a new column

df_split = df_pri.copy() 
df_split['genres_split'] = df_split['genres'].str.split(',')

# 'exploding' the 'genres_split' list to get multiple entries for each film by genre

df_split = df_split.explode('genres_split')

Now that the genre column has been cleaned to be more useful, it is possible to begin aggregate processes to explore the data.

In [ ]:
# calculating the mean value by genre
df_genres = df_split.groupby('genres_split').mean().drop('start_year', axis=1)
df_genres = df_genres.sort_values('total_gross', ascending=False)
df_genres.reset_index(inplace=True)
df_genres.head()

In [ ]:
fig, [ax1, ax2, ax3] = plt.subplots(1, 3, figsize=(20,10))
sns.barplot(data=df_genres.head(10), x='genres_split', y='domestic_gross',
            ci=68, ax=ax1)
ax1.set_title('Top 10 Domestic Gross Sales by Genre', y=1.1)
ax1.set_xlabel('Genres')
ax1.set_ylabel('Mean Gross, in 100M USD')
ax1.set_xticklabels(ax1.get_xticklabels(),rotation=45, ha='right')

sns.barplot(data=df_genres.head(10), x='genres_split', y='foreign_gross', 
            ci=68, ax=ax2)
ax2.set_title('Top 10 Foreign Gross Sales by Genre', y=1.1)
ax2.set_xlabel('Genres')
ax2.set_ylabel('Mean Gross, in 100M USD')
ax2.set_xticklabels(ax2.get_xticklabels(),rotation=45, ha='right')

sns.barplot(data=df_genres.head(10), x='genres_split', y='total_gross', 
            ci=68, ax=ax3)
ax3.set_title('Top 10 Gross Sales by Genre', y=1.1)
ax3.set_xlabel('Genres')
ax3.set_ylabel('Mean Gross, in 100M USD')
ax3.set_xticklabels(ax2.get_xticklabels(),rotation=45, ha='right');

plt.tight_layout()

There is a striking similarity between the spread of genres across domestic and foreign markets, suggesting that there is universal appeal for the highest grossing genres of film.

***The key take away here is that sci-fi, adventure, and animated films all perform exceptionally well at the box office. The second tier of genres are action, fantasy, and family movies. These seem like the natural genres for Microsoft to focus their energies.***

## Exploring the Correlation Between Critical and Commercial Success

In [ ]:
ratings_df = pd.read_csv(ratings_url)
prelim(ratings_df)

This table will be useful once merged with title_basics, for comparing the ratings on IMDB with the gross sales to explore that correlation.

It does not appear that any cleaning needs to be done. Every column has no nulls, and is of the expected dtype.

In [ ]:
# merging columns and sanity check
rat_gro_df = pd.merge(df_split, ratings_df, left_on='tconst', right_on='tconst')

# dropping columns that are not relevent to our investigation
df_cleaned = rat_gro_df.drop(['original_title', 'genres', 'title', 'studio', 'year'], axis=1)
df_cleaned.head()

In [ ]:
df_genres_rating = df_cleaned.groupby('genres_split').mean()
df_genres_rating = df_genres_rating.sort_values('averagerating', ascending=False)
df_genres_rating.reset_index(inplace=True)
df_genres_rating.head()

In [ ]:
fig, [ax1, ax2, ax3] = plt.subplots(1, 3, figsize=(20,8))
sns.barplot(data=df_genres_rating.head(15), x='genres_split', y='averagerating',
            ci=68, ax=ax1)
ax1.set_title('Top 15 Genres by Average Rating on IMBD', y=1.1)
ax1.set_xlabel('Genres')
ax1.set_ylabel('Average Rating on IMBD')
ax1.set_xticklabels(ax1.get_xticklabels(),rotation=45, ha='right')

sns.barplot(data=df_genres_rating.head(15), x='genres_split', y='averagerating', 
            ci=68, ax=ax2)
ax2.set_title('Top 15 Genres by Average Rating on IMBD', y=1.1)
ax2.set_xlabel('Genres')
ax2.set_ylabel('Average Rating on IMBD')
ax2.set_xticklabels(ax2.get_xticklabels(),rotation=45, ha='right')

sns.barplot(data=df_genres_rating.head(15), x='genres_split', y='averagerating', 
            ci=68, ax=ax3)
ax3.set_title('Top 15 Genres by Average Rating on IMBD', y=1.1)
ax3.set_xlabel('Genres')
ax3.set_ylabel('Average Rating on IMBD')
ax3.set_xticklabels(ax3.get_xticklabels(),rotation=45, ha='right')
plt.tight_layout()

The above plots show that the critical success of a genre, as measured by ratings on IMDB, does not mirror the success of a genre when measured by total sales.

This suggests that critical success does not always correlate with box office success.

In [ ]:
#plotting correlation between total_gross and averagerating
df_cleaned.corr(method='pearson')

The strongest correlations in the above matrix are generally obvious, and non-informative. For instance, a strong correlation between the number of votes and the domestic gross makes intuitive sense -- more people seeing a movie naturally leads to more people rating it on IMDB.

Other correlations, while weaker, may be more instructive. Below are plots of some of the more interesting correlations in this data frame:

In [ ]:
fig, [ax1, ax2, ax3] = plt.subplots(3, 1, figsize=(10, 20))
sns.regplot(data=(df_cleaned.sort_values('total_gross', ascending=False).head(500)), 
                                 x='total_gross', y='averagerating', ax=ax1)
ax1.set_title('Total Gross vs. Average Rating')
ax1.set_xlabel('Total Gross, in billions of USD')
ax1.set_ylabel('Average Rating on IMBD')

sns.regplot(data=(df_cleaned.sort_values('total_gross', ascending=False).head(500)), 
                                 x='total_gross', y='runtime_minutes', ax=ax2)
ax2.set_title('Total Gross vs. Film Length')
ax2.set_xlabel('Total Gross, in billions of USD')
ax2.set_ylabel('Film Length, in minutes')

sns.regplot(data=(df_cleaned.sort_values('total_gross', ascending=False).head(500)), 
                                 x='runtime_minutes', y='averagerating', ax=ax3)
ax3.set_title('Film Length vs. Average Rating')
ax3.set_xlabel('Film Length, in minues')
ax3.set_ylabel('Average Rating on IMBD');
plt.tight_layout()

## Examining ROI by Genre

Knowing the budget for these movies can provide a ballpark estimate on the ROI of movies, which can be then sorted by genre to give actionable data.


In [ ]:
budget_df = pd.read_csv(budget_url)
prelim(budget_df)

The budget_df contains data about the release date, movie title, production budget, and box office gross. Since we already have cleaned gross data, we will be merging only the production budget and movie name with our existing df.

Additionally we will have to clean and recast the production_budget column.

In [ ]:
# merge dfs and clean, recast budget column as float, dropping excess columns
# and sanity check
df = pd.merge(df_cleaned, budget_df, left_on='primary_title', right_on='movie')
df['production_budget'] = df['production_budget'].str.strip(
    '$').str.replace(',','').astype(float)
df = df.drop(['id', 'release_date', 'movie', 'domestic_gross_y',
              'worldwide_gross'], axis=1)
df.sort_values('production_budget', ascending=False).head()

In [ ]:
# combining the 'music' and 'musical' genres
df['genres_split'] = df['genres_split'].str.replace('Musical', 'Music')

# grouping by genre and aggregating by mean

df_avg = df.groupby('genres_split').mean()
# creating new column to represent ROI

df_avg['ROI'] = ((df_avg['total_gross'] - df_avg['production_budget']) / df_avg['production_budget']) * 100
# ordering by ROI

df_avg = df_avg.sort_values('ROI', ascending=False)
df_avg.reset_index(inplace=True)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2,figsize=(12,8))

sns.barplot(data=df_avg.head(15), x='genres_split', y="ROI", ax=ax1)
ax1.set_title('ROI by Genre')
ax1.set_xlabel('Genre')
ax1.set_xticklabels(ax1.get_xticklabels(),rotation=45, ha='right')

sns.barplot(data=df_avg.sort_values('production_budget', ascending=False).head(15),
                                    x ='genres_split', y='production_budget', ax=ax2)
ax2.set_title('Average Production Budget by Genre')
ax2.set_xlabel('Genre')
ax2.set_ylabel('Production Budget, in hundreds of million USD')
ax2.set_xticklabels(ax2.get_xticklabels(),rotation=45, ha='right');
plt.tight_layout()

In [ ]:
df_avg.groupby('genres_split')['ROI', 'production_budget'].mean().sort_values('production_budget')

In [ ]:
sns.lmplot(x="production_budget", y="ROI", data=df_avg, fit_reg=False, hue='genres_split');

The intersection of lower production budget and higher ROI (the upper left quadrant of the above chart) is a great place for Microsoft to begin its movie studio business. Lower risk and greater reward is a smart, efficient way to enter a crowded market.


## For Further Explanation

COVID-19 certainly changed the way that people consume movies. Can we generate any insights from looking at the increase in pay-per-view and streaming services during the pandemic? How valuable is that segement of the market once movie theaters open at full capacity again?

Is there a correlation between runtime and critical and/or box office success?